<a href="https://colab.research.google.com/github/Nicolaskimball/TICKER-LOOPS/blob/main/IBM_Yahoo_Loop_notebook_with_indicators_for_single_ticker_1m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q alpha_vantage mplfinance yfinance tensorflow_addons finta schedule

     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 1.2MB 16.9MB/s 
     |████████████████████████████████| 5.5MB 46.0MB/s 
     |████████████████████████████████| 153kB 52.2MB/s 
     |████████████████████████████████| 266kB 65.0MB/s 


In [ ]:
# download TA-Lib 
!wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 2>&1 >/dev/null
!tar xzf ta-lib-0.4.0-src.tar.gz 2>&1 >/dev/null

import os

os.chdir('ta-lib') # Can't use !cd in co-lab

!./configure --prefix=/usr --enable-silent-rules 2>&1 >/dev/null
!make -s 2>&1 >/dev/null
!make install 2>&1 >/dev/null

os.chdir('../')

!pip install -q TA-Lib 2>&1 > /dev/null

import talib

./configure: line 4354: /usr/bin/file: No such file or directory
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
ar: `u' modifier ignored since `D' is the default (see `U')
gen_code.c: In function ‘printFuncHeaderDoc’:
gen_code.c:3456:4: warning: format not a string literal and no format arguments [-Wformat-security]
    fprintf( out, prefix );
    ^~~~~~~


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
import tensorflow_addons as tfa
import tensorflow.keras.backend as kb
from sklearn import preprocessing
from alpha_vantage.timeseries import TimeSeries
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error
from collections import deque
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import sys
import time
import os
import shutil
import yfinance as yf
import finta
import schedule


API_KEY='H_ED__vbG_DJyjPiYKboH_Vf_G4Oq_dTkDhkPi'
API_KEY='I2C7Y6B8KB5MCA6N'
## UPDATE TICKER HERE.. 
TICKER = "spy"

# Window size or the sequence length
N_STEPS = 20
# Lookup step, 1 is the next day
LOOKUP_STEP = 5
# test ratio size, 0.2 is 20%
TEST_SIZE = 0.2

### model parameters
N_LAYERS = 5
# LSTM cell
CELL = LSTM
# 256 LSTM neurons
UNITS = 512
# 40% dropout
DROPOUT = 0.4
### training parameters
# mean squared error loss
LOSS = "msle"
OPTIMIZER = tfa.optimizers.RectifiedAdam(0.001)
BATCH_SIZE = 64
EPOCHS = 400
PATIENCE = 10

# Data loading parameters
INTERVAL = "5m"
PERIOD   = "5000m"

## DONT TOUCH ANYTHING BELOW HERE...
ticker = TICKER
FEATURE_COLUMNS = [
    "open",
    "close",
    "volume",
    "high",
    "low",
    "RSI",
    "MacD",
    "OBV",
    "HMA",
    "EMA",
]

date_now = time.strftime("%Y-%m-%d")

def load_data(ticker,
              n_steps=50,
              scale=True,
              shuffle=True,
              lookup_step=1, 
              test_size=0.2, 
              feature_columns=[
                  "close",
                  "high",
                  "low",
                  "volume",
                  "low",
                  "RSI",
                  "MacD",
                  "OBV",
                  "HMA",
                  "EMA",

]
):

    amd = yf.Ticker(ticker)
    df = amd.history(period=PERIOD, interval=INTERVAL)

    # Print the properties of the input data
    print(df.columns)
    print(f'Number of input records: {df.shape[0]:,}')
    print(f'Length of input records: {df.shape[1]:,}')

    # Dictionary 'dict' or map or hash map
    d = {'from': 'to'}

    # The data is in reverse order - put it so time moves to the right/down
    # df = df.sort_index(axis=0, ascending=True)
    # Assign new column names
    df.rename(
      columns={
         'Open'  : 'open',
         'Close' : 'close',
         'High'  : 'high',
         'Low'   : 'low',
         'Volume': 'volume',
      },
      inplace=True
    )
    print(df)

    # - Relative Strength Indicator
    real = talib.RSI(df['close'], timeperiod=14)
    df['RSI'] = real

    # Momentum Signal - Moving Average Convergence Divergence
    macd, macdsignal, macdhist = talib.MACD(
        df['close'],
        fastperiod=12,
        slowperiod=26,
        signalperiod=9
    )
    df['MacD'] = macd

    # Volume Signal - On Balance Volume
    obv = talib.OBV(
        df['close'],
        df['volume']
    )
    df['OBV'] = obv

    # Hull Moving Average
    hma = finta.TA.HMA(df)
    df['HMA'] = hma

    ema = talib.EMA(df['close'])
    df['EMA'] = ema

    # Trim the DataFrame to just those values without the initial nulls from the window functions
    first_all_cols_non_null_index = df.apply(lambda series: series.first_valid_index()).max()
    df = df[df.index > first_all_cols_non_null_index]

    # this will contain all the elements we want to return from this function
    result = {}
    # we will also return the original dataframe itself
    result['df'] = df.copy()

    # make sure that the passed feature_columns exist in the dataframe
    for col in feature_columns:
        assert col in df.columns
    if scale:
        column_scaler = {}
        # scale the data (prices) from 0 to 1
        for column in feature_columns:
            scaler = preprocessing.MinMaxScaler()
            df[column] = scaler.fit_transform(np.expand_dims(df[column].values, axis=1))
            column_scaler[column] = scaler

        # add the MinMaxScaler instances to the result returned
        result["column_scaler"] = column_scaler
    # add the target column (label) by shifting by `lookup_step`
    df['future'] = df['close'].shift(-lookup_step)
    # last `lookup_step` columns contains NaN in future column
    # get them before droping NaNs
    last_sequence = np.array(df[feature_columns].tail(lookup_step))
    # drop NaNs
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=n_steps)
    for entry, target in zip(df[feature_columns].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == n_steps:
            sequence_data.append([np.array(sequences), target])
    # get the last sequence by appending the last `n_step` sequence with `lookup_step` sequence
    # for instance, if n_steps=50 and lookup_step=10, last_sequence should be of 59 (that is 50+10-1) length
    # this last_sequence will be used to predict in future dates that are not available in the dataset
    last_sequence = list(sequences) + list(last_sequence)
    # shift the last sequence by -1
    last_sequence = np.array(pd.DataFrame(last_sequence).shift(-1).dropna())
    # add to result
    result['last_sequence'] = last_sequence
    # construct the X's and y's
    X, y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        y.append(target)
    # convert to numpy arrays
    X = np.array(X)
    y = np.array(y)
    # reshape X to fit the neural network
    X = X.reshape((X.shape[0], X.shape[2], X.shape[1]))
    # split the dataset
    result["X_train"], result["X_test"], result["y_train"], result["y_test"] = train_test_split(X, y, test_size=test_size, shuffle=shuffle)
    # return the result
    return result


def create_model(input_length, units=256, cell=LSTM, n_layers=2, dropout=0.3,
                loss="mean_absolute_error", optimizer="rmsprop"):
    model = Sequential()
    for i in range(n_layers):
        if i == 0:
            # first layer
            model.add(cell(units, return_sequences=True, input_shape=(None, input_length)))
        elif i == n_layers - 1:
            # last layer
            model.add(cell(units, return_sequences=False))
        else:
            # hidden layers
            model.add(cell(units, return_sequences=True))
        # add dropout after each layer
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="linear"))
    model.compile(loss=loss, metrics=["mean_absolute_error"], optimizer=optimizer)
    return model


def get_accuracy(model, data):
    y_test = data["y_test"]
    X_test = data["X_test"]
    y_pred = model.predict(X_test)
    y_test = np.squeeze(data["column_scaler"]["close"].inverse_transform(np.expand_dims(y_test, axis=0)))
    y_pred = np.squeeze(data["column_scaler"]["close"].inverse_transform(y_pred))
    y_pred = list(map(lambda current, future: int(float(future) > float(current)), y_test[:-LOOKUP_STEP], y_pred[LOOKUP_STEP:]))
    y_test = list(map(lambda current, future: int(float(future) > float(current)), y_test[:-LOOKUP_STEP], y_test[LOOKUP_STEP:]))
    return {
        'accuracy_percentage': accuracy_score(y_test, y_pred) * 100, 
        'mean_absolute_error': mean_absolute_error(y_test, y_pred),
        'mean_squared_error': mean_squared_error(y_test, y_pred),
        'mean_squared_log_error': mean_squared_log_error(y_test, y_pred),
        'r_squared_score': r2_score(y_test, y_pred)
    }


def predict(model, data, classification=False):
    # retrieve the last sequence from data
    last_sequence = data["last_sequence"][:N_STEPS]
    # retrieve the column scalers
    column_scaler = data["column_scaler"]
    # reshape the last sequence
    last_sequence = last_sequence.reshape((last_sequence.shape[1], last_sequence.shape[0]))
    # expand dimension
    last_sequence = np.expand_dims(last_sequence, axis=0)
    # get the prediction (scaled from 0 to 1)
    prediction = model.predict(last_sequence)
    # get the price (by inverting the scaling)
    predicted_price = column_scaler["close"].inverse_transform(prediction)[0][0]
    return predicted_price


def plot_graph(model, data):
    y_test = data["y_test"]
    X_test = data["X_test"]
    y_pred = model.predict(X_test)
    y_test = np.squeeze(data["column_scaler"]["close"].inverse_transform(np.expand_dims(y_test, axis=0)))
    y_pred = np.squeeze(data["column_scaler"]["close"].inverse_transform(y_pred))
    # last 200 days, feel free to edit that
    plt.plot(y_test[-12:], c='b')
    plt.plot(y_pred[-12:], c='r')
    plt.xlabel(INTERVAL)
    plt.ylabel("Price")
    plt.legend(["Actual Price", "Predicted Price"])
    plt.show()


def r2_score_keras(y_true, y_pred):
    """Implements the Coeffecient of Determination, R^2 or R-squared"""
    SS_res =  kb.sum(kb.square(y_true - y_pred))
    SS_tot = kb.sum(kb.square(y_true - kb.mean(y_true)))
    return (1 - SS_res / (SS_tot + kb.epsilon()))


def inverse_r2_score_keras(y_true, y_pred):
    """Implements the inverse Coeffecient of Determination, R^2 or R-squared"""
    SS_res =  kb.sum(kb.square(y_true - y_pred))
    SS_tot = kb.sum(kb.square(y_true - kb.mean(y_true)))
    return 1.000000000 / (1 - SS_res / (SS_tot + kb.epsilon()))


def new_predict(ticker, model, scalers):
    """Given a ticker and model, make predictions for a yahoo Ticker"""
    yahoo_ticker = yf.Ticker(ticker)

    yahoo_df = yahoo_ticker.history(period=PERIOD, interval=INTERVAL)
    yahoo_df['DateTimeBarStart'] = yahoo_df.index
    yahoo_df['DateTimeBarStart'] = yahoo_df['DateTimeBarStart'].dt.tz_localize(None)
    yahoo_df = yahoo_df.set_index('DateTimeBarStart')

    yahoo_df = yahoo_df.rename(
        columns={
            'Close': 'close',
            'Volume': 'volume',
        },
    )

    yahoo_df.rename(
      columns={
         'Open'  : 'open',
         'Close' : 'close',
         'High'  : 'high',
         'Low'   : 'low',
         'Volume': 'volume',
      },
      inplace=True
    )

    # - Relative Strength Indicator
    real = talib.RSI(yahoo_df['close'], timeperiod=14)
    yahoo_df['RSI'] = real

    # Momentum Signal - Moving Average Convergence Divergence
    macd, macdsignal, macdhist = talib.MACD(
        yahoo_df['close'],
        fastperiod=12,
        slowperiod=26,
        signalperiod=9
    )
    yahoo_df['MacD'] = macd

    # Volume Signal - On Balance Volume
    obv = talib.OBV(
        yahoo_df['close'],
        yahoo_df['volume']
    )
    yahoo_df['OBV'] = obv

    # Hull Moving Average
    hma = finta.TA.HMA(yahoo_df)
    yahoo_df['HMA'] = hma

    ema = talib.EMA(yahoo_df['close'])
    yahoo_df['EMA'] = ema

    # Retain only feautre columns
    yahoo_df = yahoo_df[FEATURE_COLUMNS]

    # During the trading day, drop the last, incomplete minute
    yahoo_df = yahoo_df.drop(
        yahoo_df.tail(1).index
    )

    # Must transform/scale the data from Yahoo using our Min/Max scaler for training data
    for column in FEATURE_COLUMNS:
        yahoo_df[f'scaled_{column}'] = scalers[column].transform(
            np.expand_dims(yahoo_df[column], 1)
        )

    scaled_column_names = [f'scaled_{x}' for x in FEATURE_COLUMNS]
    scaled_rename_dict  = {f'scaled_{x}': x for x in FEATURE_COLUMNS}
    scaled_yahoo_df = yahoo_df[scaled_column_names].rename(
        scaled_rename_dict
    )

    predict_df = scaled_yahoo_df.iloc[-N_STEPS:]

    # Expand the dimensions so we have an array of one time series window
    predict_ary = np.expand_dims(
        predict_df.values,
        axis=0
    )

    columns = []
    for i in range(predict_ary.shape[2]):
        columns.append(
            predict_ary[0, :, i]
        )
    column_predict_ary = np.expand_dims(np.stack(columns), axis=0)

    raw_prediction = model.predict(column_predict_ary)
    prediction = scalers['close'].inverse_transform(
        raw_prediction
    )[0][0]

    # Get the last timestamp, add the LOOKUP_STEP * 1 minute to get the predicted time
    last_dt = yahoo_df.index[-1]
    last_price = yahoo_df.iloc[-1]['close']

    return prediction, last_dt, last_price


def print_row(first=False, scalers=None):
    prediction, last_dt, last_price = new_predict(TICKER, model, scalers)

    last_iso_dt = last_dt.isoformat().replace('T', ' ')

    headers = ['DateTime', 'Price', 'Predicted DateTime', 'Predicted Price']
    if first:
        print(
            '{:<20s}    {:<10s}    {:<20s}    {:<20s}'.format(*headers)
        )

    future_iso_dt = (last_dt + timedelta(minutes=LOOKUP_STEP)).isoformat().replace('T', ' ')

    formatted_last_price = '{:<.4f}'.format(last_price)
    formatted_prediction = '{:<.4f}'.format(prediction)

    print(
        '{:<20s}    {:<10s}    {:<20s}    {:<20s}'.format(
            last_iso_dt, formatted_last_price, future_iso_dt, formatted_prediction
        )
    )


# Disable printing
def block_print():
    sys.stdout = open(os.devnull, 'w')


# Restore printing
def enable_print():
    sys.stdout = sys.__stdout__


# model name to save
model_name = f"{date_now}_{ticker}-{LOSS}-{CELL.__name__}-seq-{N_STEPS}-step-{LOOKUP_STEP}-layers-{N_LAYERS}-units-{UNITS}"

# create these folders if they does not exist
if not os.path.isdir("results"):
    os.mkdir("results")
if not os.path.isdir("logs"):
    os.mkdir("logs")
if not os.path.isdir("data"):
    os.mkdir("data")

In [ ]:
# Load or update the data the model trains on
data = load_data(ticker, N_STEPS, lookup_step=LOOKUP_STEP, test_size=TEST_SIZE, feature_columns=FEATURE_COLUMNS,)

In [ ]:
# Remove the previous model
try:
    shutil.rmtree(os.path.join('results', 'temp_model'))
except FileNotFoundError:
    pass

In [ ]:
# Train a new model
model = create_model(N_STEPS, loss=LOSS, units=UNITS, cell=CELL, n_layers=N_LAYERS,
                     dropout=DROPOUT, optimizer=OPTIMIZER)

model.summary()

# some tensorflow callbacks
checkpointer = ModelCheckpoint(os.path.join("results", model_name), save_best_only=True, verbose=1)
tensorboard = TensorBoard(log_dir=os.path.join("logs", model_name))
early_stopping = EarlyStopping(monitor='val_loss', patience=PATIENCE, verbose=1)

history = model.fit(data["X_train"], data["y_train"],
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(data["X_test"], data["y_test"]),
                    callbacks=[checkpointer, tensorboard, early_stopping],
                    verbose=1)

model_path = os.path.join("results", model_name) + ".h5"
model.save(model_path)

In [ ]:
# Load the model we already trained and evaluate it
model.load_weights(model_path)
print(len(data["df"].index))
pd.DataFrame([get_accuracy(model, data)])      

In [ ]:

# Make a prediction using the model we previously trained
model.load_weights(model_path)

# Load the latest Yahoo data
del data
data = load_data(
    ticker,
    N_STEPS,
    lookup_step=LOOKUP_STEP,
    test_size=TEST_SIZE,
    feature_columns=FEATURE_COLUMNS,
)
# print(f'Last sequence: {data["last_sequence"]}')
print(f'Data updated. Last record is: {data["df"].index[-1]}')

# predict the future price
future_price = predict(model, data)
print(f"Future price after {LOOKUP_STEP} {INTERVAL} is {future_price:.2f}$")

# Plot the graph
# plot_graph(model, data)

In [ ]:
# Print the first row with header
schedule.clear()
schedule.every(5).minutes.at(":05").do(print_row, first=False, scalers=data['column_scaler'])
# schedule.every(5).seconds.do(print_row, first=False, scalers=data['column_scaler'])

print_row(first=True, scalers=data['column_scaler'])
# Check the schedule ever second and run any scheduled events
while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
first=True
prediction, last_dt, last_price = new_predict(TICKER, model, data['column_scaler'])
prediction, last_dt, last_price

last_iso_dt = last_dt.isoformat().replace('T', ' ')

headers = ['DateTime', 'Price', 'Predicted DateTime', 'Predicted Price']
if first:
    print(
        '{:<20s}    {:<10s}    {:<20s}    {:<20s}'.format(*headers)
    )

future_iso_dt = (last_dt + timedelta(minutes=LOOKUP_STEP)).isoformat().replace('T', ' ')

formatted_last_price = '{:<.4f}'.format(last_price)
formatted_prediction = '{:<.4f}'.format(prediction)

a = \
    '{:<20s}    {:<10s}    {:<20s}    {:<20s}'.format(
        last_iso_dt, formatted_last_price, future_iso_dt, formatted_prediction
    )

In [ ]:
print(last_iso_dt)